In [116]:
def parse_formula(formula):
    literals = {}
    literal_id = 1
    clauses = []

    def get_literal_id(literal):
        nonlocal literal_id
        if literal not in literals:
            literals[literal] = literal_id
            literal_id += 1
        return literals[literal]

    for clause in formula.split(')('):
        clause = clause.replace('(', '').replace(')', '')
        if clause == '':
            clauses.append(set())  # Add an empty clause
            continue
        literals_in_clause = set()
        for literal in clause.split('|'):
            if literal.startswith('!'):
                literals_in_clause.add(-get_literal_id(literal[1:]))
            else:
                literals_in_clause.add(get_literal_id(literal))
        clauses.append(literals_in_clause)

    return clauses, {v: k for k, v in literals.items()}


In [177]:
formula = "(a|!b|c)(!c|d|!e)(!a|!b|e)(d|b)(e|a|!c)"
clauses, literals_map = parse_formula(formula)

print("clauses: ",clauses)
print("literals map: ",literals_map)
# literal=-2
# print(literals_map[-literal])

clauses:  [{1, 3, -2}, {-5, 4, -3}, {5, -1, -2}, {2, 4}, {1, -3, 5}]
literals map:  {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e'}


### Printing Purposes

In [166]:
def convert_clause_to_literals(clause, literals_map):
    return {literals_map[abs(lit)] if lit > 0 else f'!{literals_map[abs(lit)]}' for lit in clause}

# clause_ex={1,-2,3}
# literals_map_ex={1: 'a', 2: 'b', 3: 'c'}
# print(convert_clause_to_literals(clause_ex,literals_map_ex))

def print_clauses(clauses, literals_map,message):
    print(f'{message} : ', [convert_clause_to_literals(clause, literals_map) for clause in clauses])

## Unit propagataion

In [119]:
def unit_literal(clauses,literals_map,step,assignment):
    unit_clauses = [c for c in clauses if len(c) == 1]
    print_clauses(unit_clauses,literals_map,"Unit Clauses")
  
    while unit_clauses:
        step += 1
        unit_clause = unit_clauses[0]
        literal = next(iter(unit_clause))
        print("STEP= " ,step)

        # if literal is bigger than 0, it is positive and we set it to True, otherwise it is negative and we set it to False
        if literal > 0:
            print(f'The Unit Clause Chosen: ({literals_map[literal]})')
            assignment[literals_map[literal]] = True
        else:
            print(f'The Unit Clause Chosen: (!{literals_map[-literal]})')
            #  if literal is -3 or "!c" ==> literals_map[-literal] is 3 or "c"
            assignment[literals_map[-literal]] = False
            
        new_clauses = []
        for clause in clauses:
            if literal in clause:
                continue  # Clause is satisfied, skip it
            new_clause = {l for l in clause if l != -literal}
            new_clauses.append(new_clause)
        clauses = new_clauses
        print_clauses(clauses,literals_map,"New Clauses")
        unit_clauses = [c for c in clauses if len(c) == 1]
        # print(f'Unit Clauses: {unit_clauses}')
        
    print("No Unit Clauses Left")
    return clauses,step


## Pure Literal elimination

In [179]:
def pure_literal(clauses, literals_map, step, assignment):
    print("*********************************")
    print("Pure Literal Elimination")
    print_clauses(clauses, literals_map, "Clauses")
    while True:
        print(f'STEP= {step+1}')

        # Union of all literals in all clauses for finding unique literals
        literals = set()
        for clause in clauses:
            literals |= clause
        
        # find pure literals
        pure_literals = set()
        for literal in literals:
            if -literal not in literals:
                pure_literals.add(literal)
        
        if not pure_literals:  # If there are no more pure literals, exit the loop
            break
        
        #  print pure literals
        pure_literals_alpha = {literals_map[abs(literal)] if literal > 0 else '!' + literals_map[abs(literal)] for literal in pure_literals}
        # print(f'STEP= {step}')
        print(f'Pure Literals: {pure_literals_alpha}')
        for literal in pure_literals:
            if literal > 0:
                assignment[literals_map[literal]] = True
            else:
                assignment[literals_map[-literal]] = False
                
        step += 1

        new_clauses = []
        for clause in clauses:
            if not clause & pure_literals:
                new_clauses.append(clause)
        
        clauses = new_clauses
        print_clauses(clauses, literals_map, "New Clauses")
    
    return clauses, step

## literal choose for branching

In [131]:
def choose_literal(clauses):
    for clause in clauses:
        for literal in clause:
            return literal
    return None

## Check Satisfiability Function

In [169]:
def is_satisfiable(clauses, literals_map, step, assignment):
    print(f"STEP {step} - ", end="")
    print_clauses(clauses, literals_map, "Formula")
    

    #  if no clause remains the formula is satisfiable = True
    if not clauses:
        print("Final Assignment: ", assignment)
        return True
    # if there is an empty clause, the formula is unsatisfiable = False
    if any(len(clause) == 0 for clause in clauses):
        print("Final Assignment: ", assignment)
        return False

    old_clauses = clauses[:]
    clauses, step = unit_literal(clauses, literals_map, step, assignment)
    clauses, step = pure_literal(clauses, literals_map, step, assignment)
    print("*********************************")
    
    if clauses == old_clauses:
        literal = choose_literal(clauses)
        if literal is None:
            return False
        
        print(f"Branching on literal: {literals_map[abs(literal)]}")
        
        # Branch where literal is True
        print(f"Trying {literals_map[abs(literal)]} = True")
        new_clauses_true = []
        for clause in clauses:
            if literal in clause:
                continue  # Clause is satisfied, skip it
            new_clause = {l for l in clause if l != -literal}
            new_clauses_true.append(new_clause)
        assignment_true = assignment.copy()
        if literal > 0:
            assignment_true[literals_map[literal]] = True
        else:
            assignment_true[literals_map[-literal]] = False
        if is_satisfiable(new_clauses_true, literals_map, step + 1, assignment_true):
            return True

        # Branch where literal is False
        print(f"Trying {literals_map[abs(literal)]} = False")
        new_clauses_false = []
        for clause in clauses:
            if -literal in clause:
                continue  # Clause is satisfied, skip it
            new_clause = {l for l in clause if l != literal}
            new_clauses_false.append(new_clause)
        assignment_false = assignment.copy()
        if literal > 0:
            assignment_false[literals_map[literal]] = False
        else:
            assignment_false[literals_map[-literal]] = True
        return is_satisfiable(new_clauses_false, literals_map, step + 1, assignment_false)

    return is_satisfiable(clauses, literals_map, step + 1, assignment)

In [178]:
assignment = {}
is_satisfiable(clauses, literals_map, 0, assignment)

STEP 0 - Formula :  [{'c', '!b', 'a'}, {'!c', 'd', '!e'}, {'e', '!b', '!a'}, {'d', 'b'}, {'!c', 'e', 'a'}]
Unit Clauses :  []
No Unit Clauses Left
*********************************
Pure Literal Elimination
Clauses :  [{'c', '!b', 'a'}, {'!c', 'd', '!e'}, {'e', '!b', '!a'}, {'d', 'b'}, {'!c', 'e', 'a'}]
STEP= 1
Pure Literals: {'d'}
New Clauses :  [{'c', '!b', 'a'}, {'e', '!b', '!a'}, {'!c', 'e', 'a'}]
STEP= 2
Pure Literals: {'e', '!b'}
New Clauses :  []
STEP= 3
*********************************
STEP 3 - Formula :  []
Final Assignment:  {'d': True, 'e': True, 'b': False}


True